# Evaluation Runner Notebook
Use this notebook to reproduce the Gemini-backed supervisor run exactly as it appears in our documentation.

## Prerequisites
- Activate the project virtual environment (e.g., `source .venv/bin/activate`).
- Ensure `.env` is populated with `GOOGLE_API_KEY` and any other required configuration.
- Verify the repository root contains all installed dependencies (`pip install -r requirements.txt`).

## What the notebook does
1. Sets `PYTHONPATH` to include `src` so ADK imports resolve.
2. Loads key/value pairs from `.env` into the subprocess environment (without overwriting already-exported variables).
3. Invokes `scripts/run_adk_supervisor.py --verbose` with the standard prompt set to capture a full leadership briefing.
4. Prints STDOUT and STDERR so the transcript can be archived or inspected inline.

Re-run the single code cell below after quotas reset to collect a fresh transcript for evaluation artifacts.

In [ ]:
import os
import subprocess
from pathlib import Path

REPO_ROOT = Path.cwd().resolve()
env = os.environ.copy()
env.setdefault('PYTHONPATH', str(REPO_ROOT / 'src'))
env_path = REPO_ROOT / '.env'
if env_path.exists():
    for line in env_path.read_text().splitlines():
        if line.strip() and not line.startswith('#') and '=' in line:
            key, value = line.split('=', 1)
            env.setdefault(key.strip(), value.strip())

command = [
    'python',
    'scripts/run_adk_supervisor.py',
    '--verbose',
    'Give me an ops briefing: what happened overnight, what are the top risks, and what should leadership do next?',
    'Investigate prod-app-01 with the default window and summarize key log anomalies.',
    'Provide the utilization stats and risks.',
    'Draft the leadership summary and actions.'
]
result = subprocess.run(command, cwd=REPO_ROOT, env=env, capture_output=True, text=True)
print(result.stdout)
if result.stderr:
    print('--- STDERR ---')
    print(result.stderr)